Insert the transcript file 


In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq groq==0.13.0 --progress-bar off
!pip install -qqq python-dotenv==1.0.1 --progress-bar off
!pip install groq


ERROR: To modify pip, please run the following command:
c:\users\harshita\appdata\local\programs\python\python39\python.exe -m pip install -Uqqq pip --progress-bar off
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.10.1 requires anyio<4,>=3.1.0, but you have anyio 4.8.0 which is incompatible.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: c:\users\harshita\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip


In [1]:
### Nav ###

In [3]:
import pandas as pd
file_path = 'adg0008_er_2024_10_31.csv'
input_data = pd.read_csv(file_path, sep=None, engine='python')

In [4]:
input_data

,Band,Timecode,Sprecher,Transkript,Übersetzung,Hauptüberschrift,Zwischenüberschrift,Hauptüberschrift (Übersetzung),Zwischenüberschrift (Übersetzung),Registerverknüpfungen,Anmerkungen,Anmerkungen (Übersetzung)
0,1,00:00:06.00,INT_UH,[Bandgeräusche],NaN,"Bauen und Wohnen, Stahlindustrie/Ruhrgebiet, C...",NaN,NaN,NaN,90443740#90443672#90443683,NaN,NaN
1,1,00:00:07.11,IP_HB,"Also, ich erzähl Ihnen das so zunächst wie mir...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,00:00:09.05,INT_UH,[unterbricht] Wie Ihnen das ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,00:00:10.08,IP_HB,"Ich habe mich ja für nichts vorbereitet, so ir...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,00:00:12.21,INT_UH,Genau.[Bandgeräusche] [x][Pause],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8180,7,00:24:26.22,IP_HB,"Und zwar weil die ..., na, ich möchte sagen, i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8181,7,00:24:39.05,IP_HB,Und das ist Alte noch geblieben.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8182,7,00:24:41.05,INT_UH,"Ja, und hier ist das immer Stadt gewesen und n...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8183,7,00:24:44.10,INT_UH,Gut ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Check for null values in each column
null_counts = input_data.isnull().sum()
print(null_counts)

Band                                    0
Timecode                                0
Sprecher                                0
Transkript                              0
Übersetzung                          8185
Hauptüberschrift                     8041
Zwischenüberschrift                  8185
Hauptüberschrift (Übersetzung)       8185
Zwischenüberschrift (Übersetzung)    8185
Registerverknüpfungen                8041
Anmerkungen                          8185
Anmerkungen (Übersetzung)            8185
dtype: int64


(Übersetzung, Hauptüberschrift (Übersetzung), Zwischenüberschrift (Übersetzung), Anmerkungen (Übersetzung) more than 80% of values are missing so decided to drop them and Band and Timecode do not add any value to the metadata so hence decided to drop them.

In [6]:
#keeping only required column
data= input_data[['Timecode','Sprecher','Transkript' ]]

In [7]:
data

,Timecode,Sprecher,Transkript
0,00:00:06.00,INT_UH,[Bandgeräusche]
1,00:00:07.11,IP_HB,"Also, ich erzähl Ihnen das so zunächst wie mir..."
2,00:00:09.05,INT_UH,[unterbricht] Wie Ihnen das ...
3,00:00:10.08,IP_HB,"Ich habe mich ja für nichts vorbereitet, so ir..."
4,00:00:12.21,INT_UH,Genau.[Bandgeräusche] [x][Pause]
...,...,...,...
8180,00:24:26.22,IP_HB,"Und zwar weil die ..., na, ich möchte sagen, i..."
8181,00:24:39.05,IP_HB,Und das ist Alte noch geblieben.
8182,00:24:41.05,INT_UH,"Ja, und hier ist das immer Stadt gewesen und n..."
8183,00:24:44.10,INT_UH,Gut ...


In [8]:
# Convert Timecode to a standard time format (assuming it's in HH:MM:SS)
data['Timecode'] = pd.to_datetime(data['Timecode'], errors='coerce').dt.time

# Chunking logic
chunked_data = []
current_speaker = None
current_start_time = None
current_text = ""

for index, row in data.iterrows():
    speaker = row['Sprecher']
    timecode = row['Timecode']
    transcript = row['Transkript']

    if speaker != current_speaker:
        # Save previous chunk if exists
        if current_speaker is not None:
            chunked_data.append({
                'Speaker': current_speaker,
                'Start Time': current_start_time,
                'End Time': prev_timecode,  # End time is the last timecode of the chunk
                'Transcript': current_text.strip()
            })
        
        # Start a new chunk
        current_speaker = speaker
        current_start_time = timecode
        current_text = transcript
    else:
        # Continue appending to the same speaker's chunk
        current_text += " " + transcript

    prev_timecode = timecode  # Store last timecode for end time

# Save the last chunk
if current_speaker is not None:
    chunked_data.append({
        'Speaker': current_speaker,
        'Start Time': current_start_time,
        'End Time': prev_timecode,
        'Transcript': current_text.strip()
    })

# Convert to DataFrame
chunked_df = pd.DataFrame(chunked_data)

/tmp/ipykernel_54/2419219159.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Timecode'] = pd.to_datetime(data['Timecode'], errors='coerce').dt.time
/tmp/ipykernel_54/2419219159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Timecode'] = pd.to_datetime(data['Timecode'], errors='coerce').dt.time


In [9]:
chunked_df

,Speaker,Start Time,End Time,Transcript
0,INT_UH,00:00:06,00:00:06,[Bandgeräusche]
1,IP_HB,00:00:07.110000,00:00:07.110000,"Also, ich erzähl Ihnen das so zunächst wie mir..."
2,INT_UH,00:00:09.050000,00:00:09.050000,[unterbricht] Wie Ihnen das ...
3,IP_HB,00:00:10.080000,00:00:10.080000,"Ich habe mich ja für nichts vorbereitet, so ir..."
4,INT_UH,00:00:12.210000,00:00:12.210000,Genau.[Bandgeräusche] [x][Pause]
...,...,...,...,...
4198,INT_UH,00:24:02.190000,00:24:16.240000,Uns interessiert ja eigentlich so der Alltag s...
4199,IP_HB,00:24:23.100000,00:24:23.100000,Also hat ihnen meine Frau genau dargelegt.
4200,INT_UH,00:24:25.160000,00:24:25.160000,"Ja, ja, das konnte man daran gut (???)"
4201,IP_HB,00:24:26.220000,00:24:39.050000,"Und zwar weil die ..., na, ich möchte sagen, i..."


Transcript was reduced from 8185 rows → 4203 rows, it means that chunking successfully merged consecutive speaker turns, making the data cleaner and more structured

In [10]:
print(chunked_df['Speaker'].value_counts())

Speaker
INT_UH    2102
IP_HB     2101
Name: count, dtype: int64


In [17]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

# Function to count tokens (approximate, assuming 1 word ≈ 1.2 tokens)
def count_tokens(text):
    return len(text.split()) * 1.2  # Rough estimate

# Parameters
MIN_TOKENS = 256  # Ensure each chunk has at least 256 tokens
MAX_TOKENS = 512  # Prevent exceeding model constraints

# Initialize list for final merged chunks
merged_chunks = []
temp_chunk = []
temp_token_count = 0
start_time = None
speaker = None

# Process each chunk row
for _, row in sentence_chunked_df.iterrows():
    sentence = row['Transcript']
    sentence_tokens = count_tokens(sentence)

    # If adding this chunk keeps us within MAX_TOKENS
    if temp_token_count + sentence_tokens <= MAX_TOKENS:
        if not temp_chunk:
            start_time = row['Start Time']  # Start new chunk
            speaker = row['Speaker']
        temp_chunk.append(sentence)
        temp_token_count += sentence_tokens
    else:
        # Save the previous chunk before starting a new one
        if temp_chunk:
            merged_chunks.append({
                'Speaker': speaker,
                'Start Time': start_time,
                'End Time': row['End Time'],
                'Transcript': " ".join(temp_chunk)
            })

        # Start a new chunk with the current sentence
        temp_chunk = [sentence]
        temp_token_count = sentence_tokens
        start_time = row['Start Time']
        speaker = row['Speaker']

# Save last chunk if any content remains
if temp_chunk:
    merged_chunks.append({
        'Speaker': speaker,
        'Start Time': start_time,
        'End Time': row['End Time'],
        'Transcript': " ".join(temp_chunk)
    })

# Convert to DataFrame
final_merged_df = pd.DataFrame(merged_chunks)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
print(final_merged_df['Transcript'].apply(lambda x: len(x.split())).describe())

count    197.000000
mean     392.015228
std       52.146035
min       61.000000
25%      383.000000
50%      410.000000
75%      419.000000
max      512.000000
Name: Transcript, dtype: float64


Metric	Value	Meaning
Total Chunks	197	
Mean (Avg. tokens per chunk)	392	
Min (Smallest chunk)	61	
25% (1st quartile)	383 tokens	Most chunks are long enough
50% (Median chunk size)	410 tokens	
75% (3rd quartile)	419 tokens	Majority are optimally sized
Max (Largest chunk)	512 tokens	No chunk exceeds the AI model's token limit 

In [7]:
from groq import Groq
import pandas as pd 

# Directly set the API key
api_key = "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" 

# Initialize the Groq client
client = Groq(api_key=api_key)

print("Groq client initialized successfully!")

Groq client initialized successfully!


In [5]:
### Dhar ###
### Choose 1 model###

In [4]:
import pandas as pd

def extract_metadata_from_chunks(client, model_name: str, chunks: list) -> pd.DataFrame:
    """
    Extract metadata from multiple chunks of a German transcript using the specified model.
    
    Args:
        client: The Groq client object.
        model_name: The name of the model to use (e.g., 'llama3-8b-8192').
        chunks: A list of transcript chunks to analyze.
    
    Returns:
        A DataFrame containing the extracted metadata for all chunks.
    """
    all_metadata = []

    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}...")
        
        # Prompt for metadata extraction
        prompt = f"""
        Extrahieren Sie die folgenden Informationen aus dem Transkript und geben Sie die Antwort auf Deutsch:

        ### Persönliche Informationen:
        1. NAME: Der vollständige Name der Person (Vor- und Nachname, falls verfügbar).
        2. JAHRGANG: Das Geburtsjahr der Person.
        3. ORT: Der Geburtsort der Person.
        4. GESCHLECHT: Das Geschlecht der Person (z.B. männlich, weiblich, divers).
        5. BERUF: Der aktuelle Beruf der Person.

        ### Informationen über Vater und Partner:
        6. VAT_JG: Geburtsjahr des Vaters.
        7. VAT_KONFESSION: Religion des Vaters.
        8. VAT_HERKUN: Herkunft des Vaters.
        9. VAT_SCHULE: Schulbildung des Vaters.
        10. VAT_AUSBIL: Ausbildung des Vaters.
        11. VAT_STAND: Beruflicher Status des Vaters.
        12. VAT_POLOR: Politische Orientierung des Vaters.

        Transkript:
        {chunk}

        Metadaten:
        NAME:
        JAHRGANG:
        ORT:
        GESCHLECHT:
        BERUF:
        VAT_JG:
        VAT_KONFESSION:
        VAT_HERKUN:
        VAT_SCHULE:
        VAT_AUSBIL:
        VAT_STAND:
        VAT_POLOR:
        """
        
        # Generate response using the Groq client
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        
        # Extract metadata from the response
        metadata = response.choices[0].message.content

        # Parse metadata into a DataFrame
        numbered_lines = [
            line for line in metadata.split("\n") 
            if line.strip().startswith(tuple(f"{i}." for i in range(1, 13)))
        ]

        for line in numbered_lines:
            if ":" in line:
                key, value = line.split(":", 1)
                key = key.strip()
                value = value.strip()
                if key in combined_metadata:
                    combined_metadata[key] += f" | {value}"  # Concatenate values with a separator
                else:
                    combined_metadata[key] = value
    
    # Convert the combined metadata into a single-row DataFrame
    final_metadata = pd.DataFrame([combined_metadata])
    return final_metadata
        
    #     data = [
    #         line.split(":", 1) if ":" in line else (line, "")  # Handle cases with no details
    #         for line in numbered_lines
    #     ]
        
    #     # Convert the chunk's metadata to a DataFrame
    #     df_chunk = pd.DataFrame(data, columns=["Category", "Details"])
        
    #     # Ensure clean columns
    #     df_chunk["Category"] = df_chunk["Category"].str.strip()
    #     df_chunk["Details"] = df_chunk["Details"].str.strip()
        
    #     # Transpose to make categories columns
    #     df_transposed = pd.DataFrame([df_chunk["Details"].values], columns=df_chunk["Category"].values)
    #     df_transposed["Chunk_ID"] = i + 1  # Add Chunk ID for reference
        
    #     # Append to the list
    #     all_metadata.append(df_transposed)
    
    # # Combine all chunk results into a single DataFrame
    # final_metadata = pd.concat(all_metadata, ignore_index=True)
    # return final_metadata


In [9]:
# Define the models
MODEL_LLAMA_70B = "llama-3.3-70b-versatile"
MODEL_LLAMA_70B_specdec = "llama-3.3-70b-specdec"

# Extract metadata for all chunks using both models
print("Processing with MODEL_LLAMA_70B...")
llama_70b_responses = extract_metadata_from_chunks(client, MODEL_LLAMA_70B, cleaned_chunks)

print("Processing with MODEL_LLAMA_70B_specdec...")
llama_70b_spec_responses = extract_metadata_from_chunks(client, MODEL_LLAMA_70B_specdec, cleaned_chunks)

# Display responses
print("Llama 70B Metadata Results:")
print(llama_70b_responses)

print("Llama 70B Spec Metadata Results:")
print(llama_70b_spec_responses)

# Save results to CSV (optional)
llama_70b_responses.to_csv("llama_70b_metadata_results.csv", index=False)
llama_70b_spec_responses.to_csv("llama_70b_spec_metadata_results.csv", index=False)


Processing with MODEL_LLAMA_70B...
Processing chunk 1/49...
Processing chunk 2/49...
Processing chunk 3/49...
Processing chunk 4/49...
Processing chunk 5/49...
Processing chunk 6/49...
Processing chunk 7/49...
Processing chunk 8/49...
Processing chunk 9/49...
Processing chunk 10/49...
Processing chunk 11/49...
Processing chunk 12/49...
Processing chunk 13/49...
Processing chunk 14/49...
Processing chunk 15/49...
Processing chunk 16/49...
Processing chunk 17/49...
Processing chunk 18/49...
Processing chunk 19/49...
Processing chunk 20/49...
Processing chunk 21/49...
Processing chunk 22/49...
Processing chunk 23/49...
Processing chunk 24/49...
Processing chunk 25/49...
Processing chunk 26/49...
Processing chunk 27/49...
Processing chunk 28/49...
Processing chunk 29/49...
Processing chunk 30/49...
Processing chunk 31/49...
Processing chunk 32/49...
Processing chunk 33/49...
Processing chunk 34/49...
Processing chunk 35/49...
Processing chunk 36/49...
Processing chunk 37/49...
Processing c

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-specdec` in organization `org_01jgv12bvtexksmf5b91w65bfx` service tier `on_demand` on : Limit 100000, Used 100772, Requested 2673. Please try again in 49m37.143s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [10]:
llama_70b_responses

,1. **NAME**,2. **JAHRGANG**,3. **ORT**,4. **GESCHLECHT**,5. **BERUF**,6. **VAT_JG**,7. **VAT_KONFESSION**,8. **VAT_HERKUN**,9. **VAT_SCHULE**,10. **VAT_AUSBIL**,...,3. ORT,4. GESCHLECHT,5. BERUF,6. VAT_JG,7. VAT_KONFESSION,8. VAT_HERKUN,9. VAT_SCHULE,10. VAT_AUSBIL,11. VAT_STAND,12. VAT_POLOR
0,Der Name der Person wird nicht explizit im Tra...,Der Jahrgang der Person wird nicht direkt gena...,Die Person ist in Altendorf geboren und hat do...,Das Geschlecht der Person wird nicht explizit ...,Die Person hat bei der Industrie- und Handelsk...,Das Geburtsjahr des Vaters wird nicht genannt.,Die Konfession des Vaters wird nicht erwähnt.,Die Herkunft des Vaters wird nicht spezifizier...,Die Schulbildung des Vaters wird nicht genannt.,Die Ausbildung des Vaters wird als Former besc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Leider nicht explizit im Transkript erwähnt.,"1910 (berechnet aus dem Jahr 1933, in dem die ...","Der genaue Geburtsort wird nicht erwähnt, aber...",Männlich (impliziert durch die Erzählung und d...,"Zu Beginn des Transkripts wird erwähnt, dass d...",Nicht explizit erwähnt.,Evangelisch (da die Familie evangelisch war).,Ostpreußen.,Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Leider nicht explizit im Transkript erwähnt.,"Nicht direkt erwähnt, aber es wird von der Zei...","Der genaue Geburtsort wird nicht erwähnt, aber...","Männlich, basierend auf der Erzählung und der ...",Nicht explizit erwähnt.,Nicht erwähnt.,"Es wird erwähnt, dass der Vater aus einer länd...","Der Vater kam aus einer ländlichen Gegend, abe...",Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Leider nicht explizit im Transkript erwähnt.,"Der Sprecher erwähnt, dass er im Ersten Weltkr...","Der Sprecher erwähnt Orte wie Frintrop, Himmel...","Männlich, da der Sprecher sich selbst als ""Jun...","Der Sprecher erwähnt, dass er zur Firma Krupp ...",Nicht explizit erwähnt.,"Der Sprecher erwähnt, dass seine Eltern ""treu ...",Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Der vollständige Name der Person wird nicht ex...,Das Geburtsjahr der Person wird nicht direkt g...,Der Geburtsort der Person wird nicht explizit ...,Die Person spricht über ihre Erfahrungen in de...,Der aktuelle Beruf der Person wird nicht im Tr...,Das Geburtsjahr des Vaters wird nicht genannt.,Die Religion des Vaters wird nicht explizit er...,Die Herkunft des Vaters wird nicht genannt.,Die Schulbildung des Vaters wird nicht erwähnt.,Die Ausbildung des Vaters wird nicht genannt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Leider nicht explizit im Transkript erwähnt.,"Nicht direkt erwähnt, aber es wird erwähnt, da...","Der Sprecher erwähnt ""Altessen"" und ""Essen"" al...","Männlich, da der Sprecher sich selbst als ""ich...","Der Sprecher erwähnt, dass er bei der Handelsk...",Nicht erwähnt.,"Der Sprecher erwähnt ""Methodisten"" und ""evange...",Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Leider nicht explizit im Transkript genannt.,Nicht genannt.,Nicht genannt.,"Männlich, basierend auf der Erzählung und der ...","Der Befragte war in der Verwaltung tätig, spez...",Nicht genannt.,Nicht genannt.,Nicht genannt.,Nicht genannt.,Nicht genannt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Leider nicht explizit im Transkript erwähnt.,Nicht erwähnt.,Der Sprecher erwähnt Essen als seinen aktuelle...,"Männlich, basierend auf der Sprache und den Pr...","Der Sprecher erwähnt, dass er bei Krupp gearbe...",Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Leider nicht explizit im Transkript erwähnt.,Nicht direkt im Transkript genannt.,"Der Sprecher erwähnt mehrere Orte, darunter Di...","Männlich, basierend auf der Erzählung und der ...",Der Sprecher war Verwaltungsmann bei Krupp.,Nicht im Transkript erwähnt.,Keine Information.,Keine Information.,Keine Information.,Keine Informati